<a href="https://colab.research.google.com/github/ChuchoDC/shape_red/blob/main/ShapeRed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ShapeRed

A continuación, se muestra de manera interactiva como ejecutar el programa adecuadamente.

Primero, es necesario cambiar el entorno de ejecución, ya que por defecto se utiliza CPU, lo cual hará que el código tarde más tiempo en ejecutarse, esto se puede realizar como se muestra en la imagen, donde vamos a seleccionar un entorno con GPU. 

![ManualGPU](../images/ManualGPU.png)

Una vez cambiado el entorno, vamos a ejecutar la celda que se encuentra debajo, esto hará que el programa se conecte con google drive y pueda realizar la lectura y escritura de archivos.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

En la siguiente celda unicamente se realiza la importación de las librerias necesarias para el funcionamiento del programa.

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau

A continuación, se define una función necesaria para cargar los datos que se van a utilizar para el entrenamiento:

In [ ]:
def cargar_datos(csv_file, images_folder, img_size):
    datos = pd.read_csv(csv_file)
    imagenes = []
    landmarks = []

    for _, row in datos.iterrows():
        img_path = os.path.join(images_folder, row['id'])
        img = cv2.imread(img_path)
        if img is None:
            print(f'Advertencia: No se pudo cargar la imagen {row["id"]}.')
            continue

        original_size = (img.shape[1], img.shape[0])
        img_resized = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        img_normalized = img_resized / 255.0
        imagenes.append(img_normalized)

        coords = row[1:].values.astype(np.float32)
        coords[::2] /= original_size[0]
        coords[1::2] /= original_size[1]
        landmarks.append(coords)

    return np.array(imagenes), np.array(landmarks)

En el siguiente bloque es necesario modificar el valor de `csv_file` además de `images_folder`, para esto, es necesario copiar la ruta tanto de nuestro archivo **csv** como de la carpeta con las imagenes asociadas.

Para esto, es necesario seleccionar la carpeta `Archivos` $\rightarrow$ `drive` $\rightarrow$ `MyDrive` y buscar tanto la carpeta como el csv, copiar la ruta de acceso y pegarla en donde se indica.

![Manual02](../images/Manual02.PNG)

Ya que tenemos la ruta de los archivos se ejecuta la siguiente celda, en caso de que no tengamos la misma cantidad de imágenes y landmarks el programa va a mostrar una advertencia, si es el caso es necesario revisar que los archivos sean correctos.

In [ ]:
imagenes, landmarks = cargar_datos(
    csv_file = 'Archivo_csv',    # Aquí se coloca la ruta del archivo csv
    images_folder = 'DirectorioDeImagenes',    # Aquí se coloca la ruta de la carpeta con imagenes
    img_size = (100, 100)
)
if imagenes.shape[0] != landmarks.shape[0]:
  print("Advertencia, no hay el mismo número de Landmarks e imágenes")

print(f"Imágenes cargadas: {imagenes.shape}")
print(f"Landmarks cargados: {landmarks.shape}")

En la siguiente celda, se define la arquitectura del modelo que se utiliza.

In [ ]:
def modelo(input_shape, num_landmarks):
    model = Sequential([
        Conv2D(32, (7, 7), activation='relu', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Conv2D(64, (5, 5), activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.3),

        Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(2, 2),
        Dropout(0.4),

        Conv2D(256, (2, 2), activation='relu', kernel_regularizer=l2(0.01)),
        Flatten(),
        Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.5),
        Dense(num_landmarks * 2, activation='sigmoid')
    ])
    return model

A continuación, es necesario colocar el número de landmarks con el que estamos trabajando. Además, es en la siguiente celda donde se va a comenzar el entrenamiento, por lo que puede tardar cierto tiempo en ejecutarse.

In [ ]:
img_size = (100, 100)
input_shape = (img_size[0], img_size[1], 3)
num_landmarks = 0          # Colocar el número de landmarks con los que se cuenta

modelo = modelo(input_shape, num_landmarks)
modelo.compile(optimizer = Nadam(learning_rate=0.0005), loss="mae")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-7)
historial = modelo.fit(imagenes, landmarks, epochs=120, batch_size=4, validation_split=0.25, callbacks=[reduce_lr])

modelo.save('modeloPropuesto.h5')

Una vez realizado el entrenamiento, se va a ejecutar la siguiente celda, la cual va a definir una función para la predicción de landmarks en imágenes nuevas.

In [ ]:
def predecir_landmarks(modelo, imagenes, output_csv, img_size=(100, 100)):
    predicciones = []

    for img_name in os.listdir(imagenes):
        img_path = os.path.join(imagenes, img_name)
        img = cv2.imread(img_path)

        if img is None:
            print(f'Advertencia: No se pudo cargar la imagen {img_name}.')
            continue

        original_size = (img.shape[1], img.shape[0])
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA)
        img = img / 255.0
        img = np.expand_dims(img, axis=0)

        pred = modelo.predict(img, verbose=0)[0]

        pred[::2] *= original_size[0]
        pred[1::2] *= original_size[1]

        predicciones.append([img_name] + pred.tolist())

    columnas = ['id'] + [f'X{i}' for i in range(len(pred) // 2)] + [f'Y{i}' for i in range(len(pred) // 2)]
    resultados = pd.DataFrame(predicciones, columns=columnas)
    resultados.to_csv(output_csv, index=False)
    print(f'Resultados guardados en {output_csv}')

Por último, tal como se hizo al inicio, es necesario copiar la ruta de la carpeta con las imágenes nuevas y colocarla para que el programa se encarge de la predicción de los landmarks, aquí se va a generar un archivo nuevo el cual contenga las coordenadas para las imágenes.

In [ ]:
carpeta_imagenes = 'Imagenes Nuevas'
output_csv = 'CNN.csv'

modelo = load_model('modeloPropuesto.h5', custom_objects={'mae': MeanAbsoluteError()})
predecir_landmarks(modelo, carpeta_imagenes, output_csv)